//////////////////////////////////////////////////
////////Damiani Volatmeter
//////////////////////////////////////////////////

dv_group = " Damiani Volatmeter "
int vis_atr = input.int(13,title="Vis ATR", group=dv_group, inline = '1')
int vis_std = input.int(20,title="Vis STD", group=dv_group, inline = '1')
int sed_atr = input.int(40,title="Sed ATR", group=dv_group, inline = '1')
int sed_std = input.int(100,title="SEd STD", group=dv_group, inline = '1')
float threshold_level = input.float(1.4,title="Threshold", group=dv_group, inline = '1')
bool lag_supressor = input.bool(true,title="Lag Supressor", group=dv_group, inline = '1')
lag_s_K = 0.5


vol = 0.0
s1_pivot=nz(vol[1], 0)
s3_pivot=nz(vol[3], 0)

vol := lag_supressor ? ta.atr(vis_atr) / ta.atr(sed_atr) + lag_s_K*(s1_pivot-s3_pivot) : ta.atr(vis_atr) / ta.atr(sed_atr)
anti_thres = ta.stdev(close, vis_std) / ta.stdev(close, sed_std)
t = threshold_level - anti_thres
vol_m = vol > t ? -1 : 0.03

In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
from legitindicators import damiani_volatmeter
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
# Load data and check if file exists
import os

if os.path.exists('SPY.csv'):
    df = pd.read_csv('SPY.csv')
    print(f"Data loaded. Shape: {df.shape}")
    print("\nFirst few rows:")
    rawdf = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    print(df.head())
else:
    print("SPY.csv file not found. Downloading data from Yahoo Finance...")
    import yfinance as yf
    df = yf.download('SPY', start='2010-01-01', end='2025-10-22', multi_level_index=False)
    df.reset_index(inplace=True)
    df.to_csv('SPY.csv', index=False)
    rawdf = df.copy()
    print(f"\nData downloaded and saved. Shape: {df.shape}")
    print("\nFirst few rows:")
    print(df.head())

Data loaded. Shape: (3976, 6)

First few rows:
        Date      Close       High        Low       Open     Volume
0 2010-01-04  85.279205  85.324353  83.909682  84.556820  118944600
1 2010-01-05  85.504936  85.542563  84.917998  85.226520  111579900
2 2010-01-06  85.565147  85.775842  85.354452  85.422173  116074400
3 2010-01-07  85.926346  86.031693  85.166334  85.407129  131091100
4 2010-01-08  86.212257  86.249884  85.527499  85.700567  126402800


In [3]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]


In [4]:
# Define Damiani Volatmeter parameters
vis_atr = 13  # Visual ATR period
vis_std = 20  # Visual STD period
sed_atr = 40  # Sediment ATR period
sed_std = 100  # Sediment STD period
threshold_level = 1.4  # Threshold level
lag_supressor = True  # Lag suppressor flag
lag_s_K = 0.5  # Lag suppressor coefficient

# Calculate ATR for both periods
def calculate_atr(high, low, close, period):
    tr = pd.DataFrame()
    tr['hl'] = high - low
    tr['hc'] = abs(high - close.shift(1))
    tr['lc'] = abs(low - close.shift(1))
    tr['tr'] = tr[['hl', 'hc', 'lc']].max(axis=1)
    return tr['tr'].rolling(window=period, min_periods=1).mean()

# Make a copy of the dataframe to avoid modifying the original
calc_df = df.copy()

# Initialize volatmeter series
vol = pd.Series(0.0, index=calc_df.index)

# Calculate ATR ratios
vis_atr_values = calculate_atr(calc_df['High'], calc_df['Low'], calc_df['Close'], vis_atr)
sed_atr_values = calculate_atr(calc_df['High'], calc_df['Low'], calc_df['Close'], sed_atr)

# Calculate standard deviation ratios with min_periods=1 to avoid initial NaNs
vis_std_values = calc_df['Close'].rolling(window=vis_std, min_periods=1).std()
sed_std_values = calc_df['Close'].rolling(window=sed_std, min_periods=1).std()

# Avoid division by zero in standard deviation ratio
sed_std_values = sed_std_values.replace(0, np.nan)
anti_thres = vis_std_values / sed_std_values
anti_thres = anti_thres.fillna(0)

# Calculate Volatmeter
for i in range(3, len(calc_df)):
    s1_pivot = vol.iloc[i-1] if i > 0 else 0
    s3_pivot = vol.iloc[i-3] if i > 2 else 0
    
    atr_ratio = vis_atr_values.iloc[i] / sed_atr_values.iloc[i] if sed_atr_values.iloc[i] != 0 else 0
    
    if lag_supressor:
        vol.iloc[i] = atr_ratio + (lag_s_K * (s1_pivot - s3_pivot))
    else:
        vol.iloc[i] = atr_ratio

# Calculate threshold and final signal
t = threshold_level - anti_thres
vol_m = pd.Series(np.where(vol > t, 'short', 'long'), index=calc_df.index)

# Add results to dataframe
df['volatmeter'] = vol
df['threshold'] = t
df['vol_signal'] = vol_m



In [5]:

# Drop NaN values and reset index
df = df.dropna().reset_index(drop=True)

df.tail()

,Date,Close,High,Low,Open,Volume,volatmeter,threshold,vol_signal
3971,2025-10-16,660.640015,668.710022,657.109985,666.820007,110563300,1.407254,1.201652,short
3972,2025-10-17,664.390015,665.760010,658.140015,659.500000,96500900,1.386439,1.200283,short
3973,2025-10-20,671.299988,672.210022,667.270020,667.320007,60493400,1.399673,1.191300,short
3974,2025-10-21,671.289978,672.989990,669.979980,671.440002,56249000,1.385264,1.183809,short
3975,2025-10-22,667.799988,671.989990,663.299988,672.000000,75552387,1.413677,1.185165,short


In [6]:
# Create a visualization of the Damiani Volatmeter
if __name__ == '__main__':

    chart = Chart(title="Damiani Volatmeter with Signals", maximize=True, inner_height=0.8)
    chart.set(df)


    # Create Volatmeter subchart
    DV_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
        
    # Plot Volatmeter and threshold on subchart
    volatmeter_line = DV_chart.create_line('Volatmeter', color="#2ee30f", width=2, price_line=False, price_label=False)
    volatmeter_line.set(df[['Date', 'volatmeter']])
    threshold_line = DV_chart.create_line('Threshold', color="red", width=2, price_line=False, price_label=False)
    threshold_line.set(df[['Date', 'threshold']])
    
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        DV_diff = df.iloc[i]['vol_signal']
        p_DV_diff = df.iloc[i-1]['vol_signal']
        current_time = df.iloc[i]['Date']

        # Check for buy signal 
        if DV_diff == "long" and p_DV_diff == "short":
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal
        elif DV_diff == "short" and p_DV_diff == "long":
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
    
chart.show(block=True)